## 네이버 뉴스 크롤링

In [5]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from pprint import pprint
from tqdm.notebook import tqdm

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RESULT_PATH = './'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url, headers={'User-Agent':'Mozilla/5.0'})
    bsoup = BeautifulSoup(breq.content)

    title = bsoup.find('h2', class_='media_end_head_headline').text
    news_detail.append(title)

    pdate = bsoup.find('span', class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text[:11]
    news_detail.append(pdate)

    _text = bsoup.find('div', class_='newsct_article _article_body').get_text().replace('\n', " ")
    _text = _text.replace("    동영상 뉴스   ", "")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.find('div', class_='copyright').find('p', class_='c_text').text
    pcompany = pcompany.split('All rights reserved.')[0].split('Copyright ⓒ')[1].strip()
    news_detail.append(pcompany)

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    crawled_data = {}
    years = []
    company = []
    title = []
    contents = []
    link = []
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    f = open(f"./contents_text_{query}.txt", 'w', encoding='utf-8-sig')
    
    for page in tqdm(range(1, maxpage_t+1, 10)):
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url,
                          headers={'User-Agent':'Mozilla/5.0'})
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont)
        
        for urls in soup.find_all("a", class_="info"):
            try :
                if urls.get_text() == "네이버뉴스":
                    print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
                    years.append(news_detail[1])
                    company.append(news_detail[4])
                    title.append(news_detail[0])
                    contents.append(news_detail[2])
                    link.append(news_detail[3])
                    
            except Exception as e:
                print(e)
                continue
        
    crawled_data["years"] = years
    crawled_data["company"] = company
    crawled_data["title"] = title
    crawled_data["contents"] = contents
    crawled_data["link"] = link
    f.close()
    return crawled_data
    
def excel_make(crawled_data):
    data = pd.DataFrame(crawled_data)
    
    xlsx_outputFileName = f'{query}_({s_date}~{e_date}).xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8-sig', index=False)


maxpage = input("최대 출력할 페이지수 입력하시오: ")
if not maxpage.isdigit():
    maxpage = input("최대 출력할 페이지수 입력하시오: ")
query = input("검색어 입력: ")
s_date = input("시작날짜 입력(ex) 2021.01.01):")
e_date = input("끝날짜 입력(ex) 2021.05.24):")
crawled_data = crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
excel_make(crawled_data) #엑셀로 만들기

최대 출력할 페이지수 입력하시오: 200
검색어 입력: 애플페이
시작날짜 입력(ex) 2021.01.01):2022.11.06
끝날짜 입력(ex) 2021.05.24):2022.12.06


  0%|          | 0/200 [00:00<?, ?it/s]

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1
https://n.news.naver.com/mnews/article/243/0000035671?sid=105
https://n.news.naver.com/mnews/article/009/0005055954?sid=101
https://n.news.naver.com/mnews/article/001/0013624816?sid=101
https://n.news.naver.com/mnews/article/003/0011576878?sid=101
https://n.news.naver.com/mnews/article/366/0000859665?sid=101
https://n.news.naver.com/mnews/article/417/0000875388?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=11
https://n.news.naver.com/mnews/article/001/0013611850?sid=101
https://n.news.naver.com/mnews/article/008/0004824141?sid=105
https://n.news.naver.com/mnews/article/016/0002074322?sid=101
https://n.news.naver.com/mnews/article/015/0004781982?sid=105
https://n.news.naver.com/mnews/article/243/0000035284?sid=101
https://n.news.na

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=181
https://n.news.naver.com/mnews/article/005/0001570072?sid=101
https://n.news.naver.com/mnews/article/658/0000026564?sid=101
https://n.news.naver.com/mnews/article/029/0002769533?sid=105
https://n.news.naver.com/mnews/article/088/0000786573?sid=105
https://n.news.naver.com/mnews/article/629/0000186841?sid=101
https://n.news.naver.com/mnews/article/277/0005184007?sid=105
https://n.news.naver.com/mnews/article/020/0003464579?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=191
https://n.news.naver.com/mnews/article/277/0005184007?sid=105
https://n.news.naver.com/mnews/article/020/0003464579?sid=101
https://n.news.naver.com/mnews/article/025/0003242129?sid=101
https://n.news.naver.com/mnews/article/031/0000713012?sid=101
https://n.news

https://n.news.naver.com/mnews/article/374/0000312488?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=391
https://n.news.naver.com/mnews/article/011/0004124809?sid=105
https://n.news.naver.com/mnews/article/009/0005052094?sid=103
https://n.news.naver.com/mnews/article/009/0005052396?sid=105
https://n.news.naver.com/mnews/article/020/0003464584?sid=101
https://n.news.naver.com/mnews/article/009/0005049675?sid=101
https://n.news.naver.com/mnews/article/016/0002067839?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=401
https://n.news.naver.com/mnews/article/050/0000063140?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=411
https://n.news.naver.com/mnews/article/374/00003

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=771
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=781
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=791
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=801
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=811
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=821
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1321
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1331
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1341
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1351
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1361
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1371
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&d

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1861
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1871
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1881
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1891
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1901
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20221106to20221206%2Ca%3A&start=1911
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.11.06&d

ModuleNotFoundError: No module named 'openpyxl'

In [2]:
import pandas as pd

data = pd.read_excel(f'data/{query}_({s_date}~{e_date}).xlsx')
data

,years,company,title,contents,link
0,2022.09.06.,중앙일보.,"""아이폰14 프로 100달러 올릴것""…삼성 안올렸는데 애플은 왜",애플이 오는 7일(현지시간) 아이폰14 시리즈를 공개할 예정이다. 아이폰14 예상 ...,https://n.news.naver.com/mnews/article/025/000...
1,2022.09.06.,ZDNet Korea.,"LT게임즈 신작 '신선놀음', 구글-애플 마켓 정식 출시",방치형 수련 RPG 장르LT게임즈 글로벌(LTGAMES GLOBAL)은 방치형 수련...,https://n.news.naver.com/mnews/article/092/000...
2,2022.09.06.,매일경제.,[단독] 여당 '애플 갑질 차단법' 세계 최초로 발의했다,수수료 폭리·광고 무임승차與 '비용전가 방지법' 마련◆ 애플 갑질 차단법 ◆ ...,https://n.news.naver.com/mnews/article/009/000...
3,2022.09.05.,연합뉴스.,"'개과천선' 애플 시리 ""한국, 독도에 확고한 영토 주권 행사""","""독도는 누구 땅"" 질문에 엉터리 답변했다가 반크 항의에 혼쭐·시정 애플 시리...",https://n.news.naver.com/mnews/article/001/001...
4,2022.09.06.,이데일리.,"'개발자 채용' 점핏, 애플 앱스토어 '오늘의 앱' 선정",[이데일리 강경래 기자] IT(정보기술) 개발자 채용 플랫폼 점핏은 애플 앱스토어 ...,https://n.news.naver.com/mnews/article/018/000...
...,...,...,...,...,...
5045,2022.04.26.,ZDNet Korea.,"애플, 노조 결성 대응 위해 반노조 로펌 소속 변호사 고용",스타벅스 노조 결성 막아온 리틀러 멘델슨 소속 변호사 고용애플이 애플스토어 노동조합...,https://n.news.naver.com/mnews/article/092/000...
5046,2022.04.26.,ZDNet Korea.,애플 아이폰14 연기설 나와...중국 봉쇄 공급망 차질 탓,현지 반도체 협력사 연기 가능성 제기코로나19 확산으로 봉쇄정책을 실시하고 있는 중...,https://n.news.naver.com/mnews/article/092/000...
5047,2022.04.26.,데일리안.,[단독] 공정위에 납작 엎드린 애플…韓 지원에 돈 ‘2배’ 썼다,공정위에 ‘동의의결 이행 결과’ 첫 제출지난해 R&D지원센터 외 계획대로 이행아이폰...,https://n.news.naver.com/mnews/article/119/000...
5048,2022.05.12.,SBS Biz.,[서학개미 뭐샀나] 애플·테슬라 급락…나스닥100 3배 추종 ETF 관심 지속,■ 경제와이드 모닝벨 '서학 개미 브리핑' - 정다인오늘(12일)도 어김없이 서학개...,https://n.news.naver.com/mnews/article/374/000...
